#### Limitations
So far, all solvers use uniform cell sizes and work only in 1D (however, `LinearSolver` seems to be almost ready for 2D simulations). Material properties are also assumed constant in the domain.

In [ ]:
using PyPlot
using LinearAlgebra
using Statistics
include("boundary_conditions.jl");

#### Define a gradient operator
That can differentiate an electric potential field to obtain an electric field.
In order to improve performance, ∇ can be called with a preallocated result and
can flip sign in the result.

In [ ]:
using StaticArrays
include("gradient.jl")

#### In-place Periodic Solver
Is aimed at solving a periodic Poisson equation without explicitely assembling the
system of linear equations. As it is a singular problem, which is non-determined
up to a constant value, the result has to be solved iteratively and some additional
constraints have to be imposed. In this case we assume that the mean value of the
solution is 0. We achieve this by substracting the original solution's mean value.

In [ ]:
include("iterative_methods.jl")
include("inplace_periodic_solver.jl")

solution(ps::InPlacePeriodicSolver) = ps.u
gradient(ps::InPlacePeriodicSolver) = let periodic = PeriodicBC()
    GradientOperator(ps.nx, ps.δx, (periodic, periodic))
end

#### Linear Solver
The former solver was limited by the fact that it worked only for periodic boundary conditions. The linear solver is aimed at solving a general Poisson equations with both periodic and Dirichlet boundary conditions. It forms a system of linear equations.

In [ ]:
using SparseArrays
include("linear_solver.jl")

solution(ps::LinearSolver) = reshape(ps.u, ps.n...)
gradient(ps::LinearSolver) = GradientOperator(ps.n[1], ps.δ[:], ps.bcs)

#### Some additional solvers
(work-in-progress)

In [ ]:
struct MutligridSolver end
struct FastFourierTransformSolver end

#### General problem parameters
The Poisson equation is solved on a domain with $x \in <0, 1>$ using a Finite Volume Method approach.
The source term is defined as $f(x) = (72\pi)^2 \sin(4\pi x) \sin(2\pi x)$.

In [ ]:
h  = 0.02
xf = 0.0:h:1.0
xc = 0.5 .* xf[1:end-1] .+
     0.5 .* xf[2:end]
φref = @.        cos(2π * 3xc)-9cos(2π * xc)
f(x) = 72.0π^2 * sin(2π * 2x) * sin(2π * x)
ρ = f.(xc)
ρ.-= mean(ρ) # substract mean value as it is a crucial condition

figure(figsize=(10, 2))
plot(xf, zero(xf), "r.-")
step(xc, ρ, where="mid")
legend(["grid", "ρ"]);

The above problem is solved using a Linear Solver. We can apply both periodic and Dirichlet boundary conditions.
In the case of boundary conditions on both boundaries we can impose additional constraint on a mean solution value. This is the reason why we substract the mean solution value after calculating the electric potential.

In [ ]:
bc = PeriodicBC()
ps = LinearSolver(xf, left=bc, right=bc)
@time it = solve!(ps, ρ)
φ  = solution(ps)
φ.-= mean(φ) # substract mean value when both boundary conditions are periodic

figure(figsize=(10, 2))
title("Solved (acc=$(norm(φ - φref)))")
plot(xf, zero(xf), "r.-")
step(xc, φ, where="mid")
plot(xc, φref, ".")
legend(["grid", "φ", "φref"]);

Once again the same Poisson equation is being solved with periodic boundary conditions on both boundaries. In this case the `solve!` procedure substract the mean value in the end.

In [ ]:
ps = InPlacePeriodicSolver(xf; maxiter=1000)
@time it = solve!(ps, ρ)
φ  = solution(ps)
φ.-= mean(φ) # substract mean value when both boundary conditions are periodic

figure(figsize=(10, 2))
title("Solved after $it iterations (acc=$(norm(φ - φref)))")
plot(xf, zero(xf), "r.-")
step(xc, φ, where="mid")
plot(xc, φref, ".")
legend(["grid", "φ", "φref"]);

Now we can also differentiate the electric potential and obtain the electric field which uses the information about boundaries.

In [ ]:
using BenchmarkTools
E = zeros(SVector{3, Float64}, length(xf));
∇ = gradient(ps)
@btime ∇($φ; result=$E)

figure(figsize=(10, 3))
plot(xf, zero(xf), "r.-")
plot(xf, 0.05E, "k-")
step(xc, φ, where="mid")
legend(["grid", "E", "φ"]);

#### Compare convergence of different methods
Comparison between iterative solvers both from Julia's IterativeSolvers.jl and our own.

In [ ]:
import IterativeSolvers

ps = LinearSolver(xf, left=bc, right=bc)
nothing

It seems that GMRES has a good convergence on the given problem

In [ ]:
x = randn(length(ps.u))
rhs = ps.b .- ρ .* h .^ 2
x, ch = IterativeSolvers.gmres!(x, ps.A, rhs, log=true)
resnorm = ch.data[:resnorm]

x = randn(length(ps.u))
@time IterativeSolvers.gmres!(x, ps.A, rhs)

figure(figsize=(10, 2))
title("GMRES from IterativeSolvers.jl")
xlabel("Iteration number")
ylabel("Residual norm")
semilogy(resnorm, linewidth=2, linestyle="-", color="k");

In [ ]:
ps = LinearSolver(xf, left=DirichletBC(), right=DirichletBC())
@time it = solve!(ps, ρ)
φ  = solution(ps)

figure(figsize=(10, 2))
plot(xf, zero(xf), "r.-")
step(xc, φ, where="mid")
legend(["grid", "φ", "φref"]);

In [ ]:
ps = LinearSolver(xf, left=DirichletBC(), right=DirichletBC())
@time it = solve!(ps, ρ)
φ  = solution(ps)

figure(figsize=(10, 2))
plot(xf, zero(xf), "r.-")
step(xc, φ, where="mid")
legend(["grid", "φ", "φref"]);

### Two-dimensional Poisson Solver
We will start with a simple example from W. Briggs, V. Henson, and S. McCormick, _Implementation_ in _A multigrid tutorial_, SIAM, 2000, ch. 4, pp. 64-68.

In [ ]:
h  = 0.02
xf = 0.0:h:1.0
yf = 0.0:h:1.0
xc = 0.5 .* xf[1:end-1] .+
     0.5 .* xf[2:end]
yc = 0.5 .* yf[1:end-1] .+
     0.5 .* yf[2:end]
nx = length(xc)
ny = length(yc)
xx = repeat(xc,   1, ny)
yy = repeat(yc', nx,  1)
f(x, y) = -2((1 - 6x^2) * y^2 * (1 - y^2) +
             (1 - 6y^2) * x^2 * (1 - x^2))
ρ = f.(xx, yy)
φref = @. (xx^2 - xx^4) * (yy^4 - yy^2)

figure(figsize=(4, 4))
imshow(ρ')
colorbar()

In [ ]:
bc = DirichletBC()
ps = LinearSolver(xf, yf, left=bc, right=bc, upper=bc, lower=bc)
@time it = solve!(ps, reshape(ρ,:))
φ  = solution(ps)
title("Solved (acc=$(norm(φ - φref)))")
imshow(φ')
colorbar();

### Let's build an electron gun!
After that, we will check if we can prescribe an electric potential inside the computational domain.

In [ ]:
h  = 0.01
xf = 0.0:h:2.0
yf = 0.0:h:0.2
xc = 0.5 .* xf[1:end-1] .+
     0.5 .* xf[2:end]
yc = 0.5 .* yf[1:end-1] .+
     0.5 .* yf[2:end]
nx = length(xc)
ny = length(yc)
xx = repeat(xc,   1, ny)
yy = repeat(yc', nx,  1)

ρ = zeros(nx, ny)
dbc = DirichletBC(0.)
hv  = DirichletBC(1000.)
bc = NeumannBC()
ps = LinearSolver(xf, yf, left=bc, right=bc, upper=bc, lower=bc)
@inbounds for i=2:19, j=10
    reset!(ps.A, ps.b, ps.dof[i, j])
    apply!(ps.A, hv,  ps.dof[i, j], ps.dof[i, j-1], ps.dof[i, j+1])
    apply!(ps.b, hv,  ps.dof[i, j])
end
@inbounds for i=2:34, j=15
    reset!(ps.A, ps.b, ps.dof[i, j])
    apply!(ps.A, dbc, ps.dof[i, j], ps.dof[i, j+1], ps.dof[i, j-1])
    apply!(ps.b, dbc, ps.dof[i, j])
end
@inbounds for i=20, j=7:10
    reset!(ps.A, ps.b, ps.dof[i, j])
    apply!(ps.A, hv,  ps.dof[i, j], ps.dof[i-1, j], ps.dof[i+1, j])
    apply!(ps.b, hv,  ps.dof[i, j])
end
@inbounds for i=35, j=5:15
    reset!(ps.A, ps.b, ps.dof[i, j])
    apply!(ps.A, dbc, ps.dof[i, j], ps.dof[i+1, j], ps.dof[i-1, j])
    apply!(ps.b, dbc, ps.dof[i, j])
end
@time it = solve!(ps, reshape(ρ,:))
φ  = solution(ps)
imshow(φ', cmap="plasma"); colorbar(orientation="horizontal")
contour(φ', colors="black", linewidths=0.8)
xlim((0, 40))
ylim((0, 15))
title("Electric potential")
nothing

In [ ]:
include("gradient.jl")
E = zeros(SVector{3, Float64}, length(xf), length(yf));
∇ = GradientOperator2(ps.n[1], ps.n[2], ps.δ[1], ps.δ[2], ps.bcs)
∇(φ; result=E);

In [ ]:
Ex = [-e[1] / norm(e) for e in E]
Ey = [-e[2] / norm(e) for e in E]
Em = [sqrt(e[1]^2 + e[2]^2) for e in E]
imshow((Em'), cmap="plasma"); colorbar(orientation="horizontal")
quiver(Ex', Ey', color="black", pivot="middle", scale=50)
xlim((0, 40))
ylim((0, 15))
title("Electric field")
nothing